## Numbers Before Matching

In [10]:
from astropy.io import fits
import numpy as np
import astropy.io.fits as pf


# Read in Redmapper data

hdul = fits.open('./Fits_files/RM_cluster.fits')
data = hdul[1].data
cols = hdul[1].columns

ra_r = data.field('RA')
dec_r = data.field('DEC')
z_r = data.field('Z')
lam_r = data.field('lambda')
jk_r = data.field('jk')

print("Number of RM Clusters after cuts: " + str(len(ra_r)))

Number of RM Clusters after cuts: 3684


In [11]:
# Read in WaZP data

W = pf.open('./Fits_files/WaZP_jk.fits')[1].data
cols = pf.open('./Fits_files/WaZP_cluster.fits')[1].columns
print('Number of WaZP Clusters before cuts: ' + str(len(W)))

# The cuts on WaZP clusters in redshift and richness/ngal
idx = np.where((W['Z']>=0.2) & (W['Z']<=0.55) & (W['lambda']>=20.) & (W['lambda']<=110.) )[0]
print('Number of WaZP Clusters after cuts: ' + str(len(idx)))

ra_u = []
dec_u = []
z_u = []
lam_u = []
jk_u = []

for val in idx:
    ra_u.append(W['ra'][val])
    dec_u.append(W['dec'][val])
    z_u.append(W['z'][val])
    lam_u.append(W['lambda'][val])
    jk_u.append(W['jk'][val])
    
ra_u = np.asarray(ra_u)
dec_u = np.asarray(dec_u)
z_u = np.asarray(z_u)
lam_u = np.asarray(lam_u)
jk_u = np.asarray(jk_u)



Number of WaZP Clusters before cuts: 58590
Number of WaZP Clusters after cuts: 7110


In [12]:
# Create an array of tuples of the form (ra, dec) for each data set
zip_radec = list(zip(ra_r, dec_r))
zip_radec_u = list(zip(ra_u, dec_u))
zip_radec = np.array(zip_radec)
zip_radec_u = np.array(zip_radec_u)


# Find indices (in **WaZP**) where RM and WaZP ra & dec are equal
idx_w = np.ndarray([0])
for val in zip_radec:
    y = np.where( (zip_radec_u[:,0] == val[0]) & (zip_radec_u[:,1] == val[1]) )[0]
    idx_w = np.append(idx_w, y) 
idx_w = idx_w.astype(int)
print(len(idx_w))

# Find indices (in **WaZP**) where RM and WaZP ra & dec are within about 5 arcseconds of each other
idx_5_arcsec = np.ndarray([0])
for val in zip_radec:
    yy = np.where( (zip_radec_u[:,0] >= val[0] - 0.0014) & (zip_radec_u[:,0] <= val[0] + 0.0014) & (zip_radec_u[:,1] >= val[1] - 0.0014) & (zip_radec_u[:,1] <= val[1] + 0.0014) )[0]
    idx_5_arcsec = np.append(idx_5_arcsec, yy)
idx_5_arcsec = idx_5_arcsec.astype(int) 
print(len(idx_5_arcsec))

1282
1346


Since the 5 arcsecond cushion only gave us an extra 64 clusters (only an increase of 5%), we decided to only use exactly matched clusters between the two catalogues.

## Creating Fits File Using WaZP Redshifts

In [16]:
# Use the indices where a match occurred to create new numpy arrays using the **WaZP** data and write it to a fits file

ra = np.ndarray([0])
dec = np.ndarray([0])
z = np.ndarray([0])
lam = np.ndarray([0])
jk = np.ndarray([0])

for i in idx_w:
    ra = np.append(ra, ra_u[i])
    dec = np.append(dec, dec_u[i])
    z = np.append(z, z_u[i])
    lam = np.append(lam, lam_u[i])
    jk = np.append(jk, jk_u[i])

# Sanity Check: Just make sure that there are no elements in our shared list that are not in both lists (return false twice if every element appears in both lists)
in_R = np.ndarray([0])
in_W = np.ndarray([0])

for i in ra:
    in_R = np.append(in_R, np.isin(i, ra_r))
    in_W = np.append(in_W, np.isin(i, ra_u))
    
print(np.isin(False, in_R))
print(np.isin(False, in_W))


ra = fits.Column(name='RA', format='E', array=ra)
dec = fits.Column(name='DEC', format= 'E', array=dec)
z = fits.Column(name='Z', format='E', array=z)
lam = fits.Column(name='LAMBDA', format='E', array = lam)
jk = fits.Column(name='JK', format='E', array=jk)

cols = fits.ColDefs([ra, dec, z, lam, jk])
hdu = fits.BinTableHDU.from_columns(cols)
#hdu.writeto('./Fits_files/Matched_WaZP_cluster.fits')

False
False


## Creating Fits File Using RM Redshifts

In [5]:
# Find indices (in **WaZP**) where RM and WaZP ra & dec are equal
idx_r = np.ndarray([0])
for val in zip_radec_u:
    y = np.where( (zip_radec[:,0] == val[0]) & (zip_radec[:,1] == val[1]) )[0]
    idx_r = np.append(idx_r, y) 
idx_r = idx_r.astype(int)

# Use the indices where a match occurred to create new numpy arrays using the **RM** data and write it to a fits file

ra = np.ndarray([0])
dec = np.ndarray([0])
z = np.ndarray([0])
lam = np.ndarray([0])
jk = np.ndarray([0])

for i in idx_r:
    ra = np.append(ra, ra_r[i])
    dec = np.append(dec, dec_r[i])
    z = np.append(z, z_r[i])
    lam = np.append(lam, lam_r[i])
    jk = np.append(jk, jk_r[i])
    
# Sanity Check: Just make sure that there are no elements in our shared list that are not in both lists (return false twice if every element appears in both lists)
in_R = np.ndarray([0])
in_W = np.ndarray([0])

for i in ra:
    in_R = np.append(in_R, np.isin(i, ra_r))
    in_W = np.append(in_W, np.isin(i, ra_u))
    
print(np.isin(False, in_R))
print(np.isin(False, in_W))

ra = fits.Column(name='RA', format='E', array=ra)
dec = fits.Column(name='DEC', format= 'E', array=dec)
z = fits.Column(name='Z', format='E', array=z)
lam = fits.Column(name='LAMBDA', format='E', array = lam)
jk = fits.Column(name='JK', format='E', array=jk)

cols = fits.ColDefs([ra, dec, z, lam, jk])
hdu = fits.BinTableHDU.from_columns(cols)
#hdu.writeto('./Fits_files/Matched_RM_cluster.fits')

False
False
